 # License
 Copyright 2020 Maciej Maślanka
 
 Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
 
 1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
 
 2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
 
 3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
 
 THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
 ****

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [57]:
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
!pip install tensorflow-datasets
import tensorflow_datasets as tfds

In [58]:
from tensorflow import keras
import numpy as np
import os
!pip install pandas
import pandas as pd
print(tf.__version__)

2.2.0-dev20200404


In [0]:
TRAIN_DATA_URL = "http://hackaton.lukask.pro//export/questions"
train_file_path = tf.keras.utils.get_file("questions.csv", TRAIN_DATA_URL)
df = pd.read_csv(train_file_path, header=None)

In [60]:
df.head()

,0,1
0,Mogę się zarazić?,1
1,Co mam zrobić?,0


In [0]:
def encode(text_tensor):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text

In [62]:
vocabulary_set = set()

for i, row in df.iterrows():
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(df.at[i,0])
    vocabulary_set.update(tokens)

vocab_size = len(vocabulary_set)
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

questions = []
answers = []

for i, row in df.iterrows():
    question = encoder.encode(df.at[i,0])
    wordNumber = len(question)
    if wordNumber < 10:
        index = wordNumber
        while index < 10:
          question.append(0)
          index += 1
    ta = tf.cast(question, tf.int64)
    ta.set_shape([None])
    questions.append(ta)
    
    tn = tf.cast(df.at[i,1], tf.int64)
    tn.set_shape([])
    answers.append(tn)
    
print(vocabulary_set)

train_data = tf.data.Dataset.from_tensor_slices(([questions],[answers]))

sample_text, sample_label= next(iter(train_data))
print(type(sample_label))
print(type(sample_text))
sample_text[0], sample_label[0]




{'mam', 'zarazić', 'się', 'zrobić', 'co', 'mogę'}
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([7, 3, 2, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

In [63]:
vocab_size += 1
print(vocab_size)

7


In [0]:
model = tf.keras.Sequential()

In [0]:
model.add(tf.keras.layers.Embedding(64,vocab_size))

In [0]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [0]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(2))

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [69]:
model.fit(train_data, epochs=3)

Epoch 1/3
1/1 [==============================] - 0s 2ms/step - loss: 0.6929 - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 0s 4ms/step - loss: 0.6916 - accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 0s 1ms/step - loss: 0.6910 - accuracy: 1.0000


In [70]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 7)           448       
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               36864     
_________________________________________________________________
dense_18 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_19 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 130       
Total params: 49,858
Trainable params: 49,858
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.save_weights('tmp/checkpoints/my_checkpoint')

In [72]:
sample_text, sample_label= next(iter(train_data))
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(64,vocab_size))
model2.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
for units in [64, 64]:
  model2.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model2.add(tf.keras.layers.Dense(2))
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model2.load_weights('tmp/checkpoints/my_checkpoint')
loss,acc = model2.evaluate(sample_text,  sample_label, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1/1 - 0s - loss: 0.6904 - accuracy: 1.0000
Restored model, accuracy: 100.00%


In [73]:
#!mkdir -p saved_model
model.save('tmp/saved_model/my_model') 

INFO:tensorflow:Assets written to: tmp/saved_model/my_model/assets
